In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
files = [
    'cone_intensities_1.csv',
    'cone_intensities_2.csv',
    'cone_intensities_3.csv'
]

dfs = [pd.read_csv(f, header=None) for f in files]
data = pd.concat(dfs, ignore_index=True)

In [ ]:
data

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,65.035328,43.081772,13.353891,99.039900,45.318401,64.757742,55.349306,67.262980,67.355869,68.177307,...,82.583447,63.441081,99.039828,69.367314,58.958661,91.924896,27.475547,93.121544,60.969029,left
1,19.829346,26.435726,24.901220,13.412936,26.289329,24.303995,26.115531,23.392672,38.751058,32.989069,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,left
2,9.666587,15.602418,7.238071,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,left
3,14.893102,22.918751,13.800269,2.128353,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,left
4,11.683424,3.504297,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,left
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10045,27.405630,16.708196,27.182446,43.697855,40.699074,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,right
10046,55.130184,56.569530,60.783690,67.336981,99.517681,64.166730,67.900353,67.065053,60.183592,78.375824,...,57.987646,58.805998,63.074779,67.118384,62.042362,51.188861,73.653535,66.891532,64.485430,left
10047,18.645815,31.080166,47.840696,30.575316,30.050700,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,left
10048,84.740608,62.902043,92.407360,52.300354,54.679022,57.313517,98.395306,42.183619,63.954448,64.392999,...,97.994105,81.306849,58.446568,61.926830,61.881969,87.641521,92.172771,47.760939,65.209113,right


In [ ]:
X = data.iloc[:, :-1].values.astype('float32')  # All but last column
y = data.iloc[:, -1].values   # Last column (labels: 'left' or 'right')

In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)  # e.g. 'left' -> 0, 'right' -> 1

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train)
y_train_tensor = torch.tensor(y_train)
X_test_tensor = torch.tensor(X_test)
y_test_tensor = torch.tensor(y_test)

# Create datasets and loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleNN(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, output_dim=2):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, hidden_dim)
        self.fc5 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        return self.fc5(x)  # No softmax, use CrossEntropyLoss


In [ ]:
model = SimpleNN(input_dim=32)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training
for epoch in range(70):  # 20 epochs
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

Epoch 1, Loss: 158.3519
Epoch 2, Loss: 103.8197
Epoch 3, Loss: 80.6404
Epoch 4, Loss: 72.6714
Epoch 5, Loss: 64.8927
Epoch 6, Loss: 60.9974
Epoch 7, Loss: 57.1472
Epoch 8, Loss: 55.7068
Epoch 9, Loss: 49.4871
Epoch 10, Loss: 47.4638
Epoch 11, Loss: 47.4366
Epoch 12, Loss: 46.2132
Epoch 13, Loss: 42.5255
Epoch 14, Loss: 46.2544
Epoch 15, Loss: 46.2710
Epoch 16, Loss: 40.8389
Epoch 17, Loss: 41.0380
Epoch 18, Loss: 42.2362
Epoch 19, Loss: 41.8392
Epoch 20, Loss: 39.0743
Epoch 21, Loss: 40.4712
Epoch 22, Loss: 38.6250
Epoch 23, Loss: 37.4873
Epoch 24, Loss: 42.2122
Epoch 25, Loss: 36.8263
Epoch 26, Loss: 34.5845
Epoch 27, Loss: 33.3445
Epoch 28, Loss: 32.1619
Epoch 29, Loss: 35.7454
Epoch 30, Loss: 36.1357
Epoch 31, Loss: 31.6856
Epoch 32, Loss: 29.2147
Epoch 33, Loss: 30.7223
Epoch 34, Loss: 28.0989
Epoch 35, Loss: 33.4047
Epoch 36, Loss: 33.1178
Epoch 37, Loss: 30.4410
Epoch 38, Loss: 25.5507
Epoch 39, Loss: 30.6657
Epoch 40, Loss: 26.0146
Epoch 41, Loss: 26.4574
Epoch 42, Loss: 24.9582

In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        _, predicted = torch.max(outputs, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 94.03%


In [ ]:
import pickle

In [ ]:
torch.save(model.state_dict(), "model_weights.pth")

# Save LabelEncoder and Scaler
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

print("Model and preprocessors saved.")

Model and preprocessors saved.
